In [89]:
import numpy as np
import cv2
from cv2 import aruco
import pandas as pd
import matplotlib.colors as mcolors
# import random

In [90]:
dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

In [91]:
def imshow(img, window_name="img"):
    cv2.startWindowThread()
    cv2.imshow(window_name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [92]:
# 軌跡データ
markers = pd.read_csv("output/test/markers.csv", header=[0, 1])
frames = len(markers)

# 動画データ
cap = cv2.VideoCapture("src/mov/test05.mp4")
fps = int(cap.get(cv2.CAP_PROP_FPS))    # 動画のfps
ret, frame = cap.read()
h, w = frame.shape[:2]

ids = pd.Series(markers.columns.levels[0])  # カラムのid列を取得
ids_mask = pd.Series(map(lambda id: str(id).isnumeric(), ids))  # 数値を真、文字列を偽とするマスクを作成
ids = ids.loc[ids_mask] # マスクを使って数値（id）のみを抜き出し

# 保存設定
fmt = cv2.VideoWriter_fourcc('m', 'p', '4', 'v') 
writer = cv2.VideoWriter('./output/tracking_test.mp4', fmt, fps, (w, h))

cpxy = ['cpx', 'cpy']
pos = [None, None]
pos_van = [None, None]
pos_lnr = [None, None]
pos_spr = [None, None]
orbits_spr = pd.DataFrame() # スプライン補完した際の軌跡
orbits_lnr = pd.DataFrame() # 線形補間した際の軌跡
orbits_van = pd.DataFrame()  # 元データの軌跡

font = cv2.FONT_HERSHEY_SIMPLEX 

for id in ids:
    # print("id :", id)
    for i, cp in enumerate(cpxy):
        pos[i] = markers[str(id), cp]
        pos_spr[i] = pos[i]
        pos_van[i] = pos[i]
        pos_lnr[i] = pos[i]
        if (frames - pos[i].isnull().sum()) <= 4:   # 検出できた座標が4以下ならnan（スプライン補完できないため）
            pos_spr[i] = [np.nan]
        else:
            # 通常データ
            pos_van[i] = pos_van[i].astype('Int64')
            # 線形補間
            pos_lnr[i] = pos_lnr[i].interpolate("ffill").interpolate("bfill").astype('int')
            # スプライン補完
            pos_spr[i] = pos_spr[i].interpolate(method="spline", order=2, limit_direction="both").astype(int)
    if ~np.isnan(pos_spr[0][0]) or ~np.isnan(pos_spr[1][0]) :   # 最初の座標がnanでなければorbits_sprに追加(pos_spr[i]が[nan]でなければ全て数値が入っていることは保証されている)
        orbits_van[id] = list(zip(pos_van[0], pos_van[1]))
        orbits_lnr[id] = list(zip(pos_lnr[0], pos_lnr[1]))
        orbits_spr[id] = list(zip(pos_spr[0], pos_spr[1]))

orbits_spr

,0,1,2,3
0,"(352, 92)","(636, 95)","(354, 282)","(627, 281)"
1,"(352, 93)","(638, 96)","(355, 283)","(627, 281)"
2,"(352, 93)","(638, 96)","(355, 283)","(627, 281)"
3,"(352, 93)","(638, 96)","(355, 283)","(627, 281)"
4,"(353, 93)","(638, 96)","(355, 283)","(628, 281)"
...,...,...,...,...
925,"(658, 272)","(401, 280)","(673, 95)","(400, 93)"
926,"(659, 272)","(401, 280)","(673, 95)","(400, 93)"
927,"(659, 272)","(401, 280)","(673, 95)","(401, 93)"
928,"(659, 272)","(402, 280)","(673, 95)","(400, 93)"


In [93]:
# 線のつなぎ方（並び替え）
order_columns = [0,1,3,2]
orbits_van = orbits_van.set_axis(order_columns, axis=1).sort_index(axis=1, ascending=True)
orbits_spr = orbits_spr.set_axis(order_columns, axis=1).sort_index(axis=1, ascending=True)
orbits_lnr = orbits_lnr.set_axis(order_columns, axis=1).sort_index(axis=1, ascending=True)

In [94]:
# 構造（線の接続関係）の定義
stractures = [[0, 1, 2, 3, 0]]
lines_from_stractures = []

# ストラクチャーに定義されているマーカーidのSeries（生のcsvだと誤検知のidが含まれるため）
stractures_ids = pd.Series(sum(stractures, [])).drop_duplicates().sort_values()

In [95]:
markers[('0', 'x')][10]

-0.1289827397337123

In [96]:
speed_x = pd.Series([], dtype=float)
speed_y = pd.Series([], dtype=float)
acceleration_x = pd.Series([], dtype=float)
acceleration_y = pd.Series([], dtype=float)

for f in range(frames):
    ret, img = cap.read()
    if not ret or cv2.waitKey(int(1000 / fps)) == 27:
        break

    cv2.putText(img, "frame: " + str(f), (10,20), font, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    # img = np.zeros((h, w, 3))

    # マーカー認識
    corners, ids, rejectedImgPoints = aruco.detectMarkers(img, dictionary)
    # マーカー可視化
    aruco.drawDetectedMarkers(img, corners, ids, (0, 255, 255))

    # # 線形補間を描画
    # lines_pos_lnr = np.array(list(orbits_lnr.iloc[f]))
    # print(lines_pos_lnr)
    # # cv2.polylines(img, [lines_pos_lnr], True, (255,0,0), 1)

    # # スプライン補完を描画
    # lines_pos_spr = np.array(list(orbits_spr.iloc[f]))
    # # cv2.polylines(img, [lines_pos_spr], True, (0,0,255), 1)
    # print([lines_pos_spr])

    # 定義した構造から線を描画（スプライン補完利用）
    positions = np.array(list(orbits_spr.iloc[f]))
    for line in stractures:
        for pos in line:
            lines_from_stractures.append(list(positions[pos]))
        # print(lines_from_stractures)
        cv2.polylines(img, np.array([lines_from_stractures]), False, (255,0,0), 2)
        lines_from_stractures = []

    # 座標から速度と加速度を求める
    if f > 1:
        for i, id in enumerate(stractures_ids):
            speed_x_current = round(((markers[(str(id), 'x')][f] - markers[(str(id), 'x')][f - 1])) / (fps / 1000), 2)
            if f < 3 : speed_x[i] = 0
            acceleration_x[i] = round((speed_x_current - speed_x[i]) / (fps / 1000), 2)
            speed_x[i] = speed_x_current
            txt = "[" + str(id) + "] "+"spd x: " + str(speed_x_current) + "m/sec | " + \
                    "acc x: " + str(acceleration_x[i]) + "m/sec^2"
            cv2.putText(img, txt, (10,40 + i * 20), font, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    

    # # 通常データを描画（<NA>が入っていれば描画しない）
    # lines_pos_van = np.array(list(orbits_van.iloc[f]))
    # count_nan = pd.Series(list(map(lambda t: pd.Series(t).isna().sum() ,lines_pos_van))).sum()
    # # if count_nan == 0:
    # #     cv2.polylines(img, [lines_pos_van], True, (0,255,255), 2)

    cv2.imshow("mov", img)
    writer.write(img)
    # print(f, lines_pos_spr)

cv2.destroyAllWindows()
writer.release()